# 第四章 分组

In [446]:
import numpy as np
import pandas as pd

## 一、分组模式及其对象
### 1.分组的一般模式
- 依据性别分组，统计全国人口寿命的平均值
- 依据季节分组，对每一个季节的温度进行标准化
- 一句班级分组，筛选出组内数学分数的平均值超过80分的班级
进行分组操作需得明确三个要素：分组依据、数据来源、操作及其返回结果。

df.groupy(分组依据)[数据来源].使用操作

In [618]:
df = pd.read_csv('D:\\datawhale\\joyful-pandas\\data\\learn_pandas.csv')
df.groupby('Gender')['Height'].median()
#按性别统计身高中位数

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

### 2.分组依据的本质
- 可以根据多个特征进行分组

In [448]:
df.groupby(['Gender','School'])['Height'].mean()

Gender  School                       
Female  Fudan University                 158.776923
        Peking University                158.666667
        Shanghai Jiao Tong University    159.122500
        Tsinghua University              159.753333
Male    Fudan University                 174.212500
        Peking University                172.030000
        Shanghai Jiao Tong University    176.760000
        Tsinghua University              171.638889
Name: Height, dtype: float64

- 可以通过复杂逻辑分组

In [449]:
condition=df.Weight>df.Weight.mean()
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

#### 【练一练】
请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值

In [450]:
condition1=df.Weight<=df.Weight.quantile(0.25)
condition2=df.Weight>=df.Weight.quantile(0.75)
df['fenduan']='normal'
df['fenduan']=df.fenduan.mask(condition1,'low')
df['fenduan']=df.fenduan.mask(condition2,'high')
df.groupby('fenduan')['Height'].mean()

fenduan
high      174.511364
low       154.119149
normal    162.465217
Name: Height, dtype: float64

群里小伙伴提供的方法如下：

In [608]:
condition1=["high" if x>df.Weight.quantile(0.75) else "medium" if x>df.Weight.quantile(0.25) else "low" for x in df.Weight]
print(condition1)

['low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'high', 'medium', 'high', 'medium', 'low', 'medium', 'medium', 'medium', 'low', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'high', 'medium', 'medium', 'high', 'medium', 'medium', 'high', 'medium', 'high', 'medium', 'medium', 'medium', 'high', 'high', 'high', 'high', 'medium', 'high', 'high', 'high', 'high', 'medium', 'high']


#### 【end】

In [451]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()
#此处的索引就是原先item中的元素
#如果传入多个序列进入 groupby ，那么最后分组的依据就是这两个序列对应行的唯一组合

a    164.823214
b    162.183051
c    162.794118
Name: Height, dtype: float64

In [452]:
 df.groupby([condition, item])['Height'].mean()

Weight   
False   a    160.350000
        b    158.288372
        c    158.645652
True    a    174.266667
        b    172.650000
        c    171.468182
Name: Height, dtype: float64

之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，最后分组的依据来自于数据来源组合的unique值

In [453]:
 df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


In [454]:
 df.groupby([df['School'], df['Gender']])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

### 3.Groupby对象
最终具体做分组操作时，所调用的方法都来自于 pandas 中的 groupby 对象，这个对象上定义了许多方法，也具有一些方便的属性。

In [455]:
gb = df.groupby(['School', 'Grade'])
gb

In [456]:
gb.ngroups#通过 ngroups 属性，可以得到分组个数

16

In [457]:
res = gb.groups#通过 groups 属性，可以返回从 组名 映射到 组索引列表 的字典
res

{('Fudan University', 'Freshman'): [15, 28, 63, 70, 73, 105, 108, 157, 186], ('Fudan University', 'Junior'): [26, 41, 82, 84, 90, 107, 145, 152, 173, 187, 189, 195], ('Fudan University', 'Senior'): [39, 46, 49, 52, 66, 77, 112, 129, 131, 138, 144], ('Fudan University', 'Sophomore'): [3, 4, 37, 48, 68, 98, 135, 170], ('Peking University', 'Freshman'): [1, 32, 35, 36, 38, 45, 54, 57, 88, 96, 99, 140, 185], ('Peking University', 'Junior'): [9, 20, 59, 72, 75, 102, 159, 183], ('Peking University', 'Senior'): [30, 86, 116, 127, 130, 132, 147, 194], ('Peking University', 'Sophomore'): [29, 61, 83, 101, 120], ('Shanghai Jiao Tong University', 'Freshman'): [0, 6, 10, 60, 114, 117, 119, 121, 141, 148, 149, 153, 184], ('Shanghai Jiao Tong University', 'Junior'): [31, 42, 50, 56, 58, 64, 85, 93, 115, 122, 143, 155, 164, 172, 174, 188, 190], ('Shanghai Jiao Tong University', 'Senior'): [2, 12, 19, 21, 22, 23, 79, 87, 89, 103, 104, 109, 123, 134, 156, 161, 165, 166, 171, 192, 197, 198], ('Shanghai 

In [458]:
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('Fudan University', 'Freshman'), ('Fudan University', 'Junior'), ('Fudan University', 'Senior'), ('Fudan University', 'Sophomore'), ('Peking University', 'Freshman'), ('Peking University', 'Junior'), ('Peking University', 'Senior'), ('Peking University', 'Sophomore'), ('Shanghai Jiao Tong University', 'Freshman'), ('Shanghai Jiao Tong University', 'Junior'), ('Shanghai Jiao Tong University', 'Senior'), ('Shanghai Jiao Tong University', 'Sophomore'), ('Tsinghua University', 'Freshman'), ('Tsinghua University', 'Junior'), ('Tsinghua University', 'Senior'), ('Tsinghua University', 'Sophomore')])

#### 【练一练】
上一小节介绍了可以通过 drop_duplicates 得到具体的组类别，现请用 groups 属性完成类似的功能

In [459]:
df.groupby(['School', 'Gender']).groups.keys()

dict_keys([('Fudan University', 'Female'), ('Fudan University', 'Male'), ('Peking University', 'Female'), ('Peking University', 'Male'), ('Shanghai Jiao Tong University', 'Female'), ('Shanghai Jiao Tong University', 'Male'), ('Tsinghua University', 'Female'), ('Tsinghua University', 'Male')])

#### 【end】

In [460]:
gb.size()#当 size 作为 DataFrame 的属性时，返回的是表长乘以表宽的大小
#但在 groupby 对象上表示统计每个组的元素个数

School                         Grade    
Fudan University               Freshman      9
                               Junior       12
                               Senior       11
                               Sophomore     8
Peking University              Freshman     13
                               Junior        8
                               Senior        8
                               Sophomore     5
Shanghai Jiao Tong University  Freshman     13
                               Junior       17
                               Senior       22
                               Sophomore     5
Tsinghua University            Freshman     17
                               Junior       22
                               Senior       14
                               Sophomore    16
dtype: int64

In [461]:
gb.get_group(('Fudan University', 'Freshman')).iloc[:3, :3]#前三行前三列
#通过 get_group 方法可以直接获取所在组对应的行，此时必须知道组的具体名字

,School,Grade,Name
15,Fudan University,Freshman,Changqiang Yang
28,Fudan University,Freshman,Gaoqiang Qin
63,Fudan University,Freshman,Gaofeng Zhao


### 4.分组的三大操作
- 每一个组返回一个标量值（平均值，中位数，组容量size等），可以聚合agg
- 每组返回一个Series，可以进行变换transform
- 每组返回整个组所在行的本身，可以进行过滤filter

## 二、聚合函数
### 1.内置聚合函数
在介绍agg之前，首先要了解一些直接定义在groupby对象的聚合函数，因为它的速度基本都会经过内部的优化，使用功能时应当优先考虑。根据返回标量值的原则，包括如下函数： max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod 。

In [462]:
gb = df.groupby('Gender')['Height']
gb.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [463]:
gb.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

#### 【练一练】
请查阅文档，明确 all/any/mad/skew/sem/prod 函数的含义

In [464]:
all(['a',(2,4),1,True])#list里面全’真‘为True，有’假‘为False

True

In [465]:
all(['a',(2,4),1,True])

True

In [466]:
all(['',(),0,False])#list里面全为’假‘为False

False

In [467]:
gb.mad()#mad(x)=median(|Xi-median(X)|)中位数绝对偏差

Gender
Female    4.088108
Male      5.394617
Name: Height, dtype: float64

In [468]:
gb.skew()#统计数据分布偏斜方向和程度
#skew(X)=E[（X-miu/sigma）**3]

Gender
Female   -0.219253
Male      0.437535
Name: Height, dtype: float64

In [469]:
gb.sem()#返回所请求轴上的平均值的额标准误差
#sem（X）=sqrt（（Xi-平均值）**2/（n-1））

Gender
Female    0.439893
Male      0.986985
Name: Height, dtype: float64

In [470]:
gb.prod()#计算所有元素的乘积

Gender
Female    4.232080e+290
Male      1.594210e+114
Name: Height, dtype: float64

#### 【end】
聚合函数当传入的数据来源包含多个列时，将按照列进行迭代计算

In [471]:
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


### 2.agg方法
- 使用多个函数

In [472]:
gb.agg(['sum', 'idxmax', 'skew'])

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

- 对特定列使用特定的聚合函数

In [473]:
gb.agg({'Height':['mean','max'], 'Weight':'count'})#构造字典的形式

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

#### 【练一练】
请使用【b】中的传入字典的方法完成【a】中等价的聚合任务

In [474]:
gb.agg({'Height':['sum', 'idxmax', 'skew'], 'Weight':['sum', 'idxmax', 'skew']})

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

#### 【end】

- 使用自定义函数

传入函数的参数是之前数据源中的列，逐列进行计算

In [475]:
gb.agg(lambda x: x.mean()-x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


#### 【练一练】
在 groupby 对象中可以使用 describe 方法进行统计信息汇总，请同时使用多个聚合函数，完成与该方法相同的功能。

In [476]:
gb.describe()

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

方法一：

In [477]:
def q25(arr):
    return arr.quantile(0.25)

def q50(arr):
    return arr.quantile(0.5)

def q75(arr):
    return arr.quantile(0.75)

res=gb.agg(['count', 'mean', 'std','min',q25,q50,q75])
res.rename(columns={'q25':'25%','q50':'50%','q75':'75%'},level=1)

Height                                                      Weight  \
        count       mean       std    min      25%    50%      75%  count   
Gender                                                                      
Female    132  159.19697  5.053982  145.4  155.675  159.6  162.825    135   
Male       51  173.62549  7.048485  155.7  168.900  173.4  177.150     54   

                                                      
             mean       std   min   25%   50%    75%  
Gender                                                
Female  47.918519  5.405983  34.0  44.0  48.0  52.00  
Male    72.759259  7.772557  51.0  69.0  73.0  78.75

方法二：（大群里小伙伴提供）

In [478]:
def quantile(n):
    def quantile_(x):
        return x.quantile(n/100)
    quantile_.__name__='{0}%'.format(n)##这里定义返回的名字
    return quantile_

methods=['count','mean','std','min',quantile(25),quantile(50),quantile(75),'max']

res=gb.agg({'Height':methods,'Weight':methods})
res=res.astype(np.float64)
res

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

#### 【end】

In [479]:
gb.describe()

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

In [480]:
def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():#s.mean是特定性别的身高或者体重的平均，s.name就相当于把整个列取出来
        res = 'Low'
    return res

gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


- 聚合结果重命名

如果想要对聚合结果的列名进行重命名，只需要将上述函数的位置改写成元组，元组的第一个元素为新的名字，第二个位置为原来的函数，包括聚合字符串和自定义函数

In [481]:
gb.agg([('range', lambda x: x.max()-x.min()), ('my_sum', 'sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

In [482]:
gb.agg({'Height': [('my_func', my_func), 'sum'],'Weight': lambda x:x.max()})

Height            Weight
       my_func      sum <lambda>
Gender                          
Female     Low  21014.0     63.0
Male      High   8854.9     89.0

另外需要注意，使用对一个或者多个列使用单个聚合的时候，重命名需要加方括号

In [483]:
gb.agg([('my_sum', 'sum')])

,Height,Weight
,my_sum,my_sum
Gender,,
Female,21014.0,6469.0
Male,8854.9,3929.0


In [484]:
gb.agg({'Height': [('my_func',my_func), ('wode','sum')],'Weight': [('range', lambda x:x.max())]})

Height          Weight
       my_func     wode  range
Gender                        
Female     Low  21014.0   63.0
Male      High   8854.9   89.0

## 三、变换和过滤
### 1.变换函数与transform方法

变换函数的返回值为同长度的序列，最常用的内置变换函数是累计函数： cumcount/cumsum/cumprod/cummax/cummin ，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作。此外在 groupby 对象上还定义了填充类和滑窗类的变换函数

In [485]:
gb.cummax().head()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0


#### 【练一练】
在 groupby 对象中， rank 方法也是一个实用的变换函数，请查阅它的功能并给出一个使用的例子。

In [486]:
gb.rank()#将序列按从小到大排序，取出排序后的索引值，若有n个相同的元素，排名相加除以n

,Height,Weight
0,58.0,47.5
1,5.0,19.0
2,50.0,54.0
3,NaN,14.5
4,27.0,31.5
...,...,...
195,21.5,47.5
196,83.0,83.0
197,21.5,42.0
198,31.0,23.0


In [487]:
s=pd.Series([7,-5,7,4,2,0,4])
s.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [488]:
gb.transform(lambda x: (x-x.mean())/x.std()).head()
#gb.agg(lambda x: (x-x.mean())/x.std())
#这样是会报错大的，因为里面涉及到x，最后传出的数也是一个与原序列一样的序列
#而agg传出标量

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631


#### 【练一练】
对于 transform 方法无法像 agg 一样，通过传入字典来对指定列使用特定的变换，如果需要在一次 transform 的调用中实现这种功能，请给出解决方案。

In [489]:
gb.transform(lambda x:(x-x.mean())/x.std() if x.name=='Height' else x.rank()).head()

,Height,Weight
0,-0.058760,47.5
1,-1.010925,19.0
2,2.167063,54.0
3,NaN,14.5
4,0.053133,31.5


#### 2.组索引与过滤
过滤在分组中是对于组的过滤，而索引是对于行的过滤

组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回 True 则会被保留， False 则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为 DataFrame 返回


- fillter

In [490]:
gb.agg(lambda x:x.shape[0])

,Height,Weight
Gender,,
Female,141.0,141.0
Male,59.0,59.0


In [491]:
gb.filter(lambda x: x.shape[0] > 100).head()#取出所有容量大于100的行
#在这个例子中就是取出female所在的行

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0


#### 【练一练】
从概念上说，索引功能是组过滤功能的子集，请使用 filter 函数完成 loc[.] 的功能，这里假设 ” . “是元素列表

In [619]:
df.head(3)

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22


In [623]:
df.loc[[0,1,3,4,8],['Grade','Weight','Transfer']].head(3)#loc要指定行索引和列索引

,Grade,Weight,Transfer
0,Freshman,46.0,N
1,Freshman,70.0,N
3,Sophomore,41.0,N


In [627]:
indices=[0,1,3,4,8]
gb = df.groupby(df.index.isin(indices))
gb.size()

False    195
True       5
dtype: int64

In [633]:
def my_condition(x):
    return x.name
gb.filter(my_condition)[['Grade','Weight','Transfer']]

,Grade,Weight,Transfer
0,Freshman,46.0,N
1,Freshman,70.0,N
3,Sophomore,41.0,N
4,Sophomore,74.0,N
8,Freshman,48.0,N


## 四、跨列分组
### 1.apply的引入和使用
在设计上， apply 的自定义函数传入参数与 filter 完全一致，只不过后者只允许返回布尔值。

In [499]:
def BMI(x):
    Height = x['Height']/100
    Weight = x['Weight']
    BMI_value = Weight/Height**2
    return BMI_value.mean()

gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

- 标量情况

In [500]:
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.apply(lambda x: 0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [501]:
gb.apply(lambda x: [0, 0]) # 虽然是列表，但是作为返回值仍然看作标量

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

In [502]:
gb.apply(lambda x: pd.Series([0,0],index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

#### 【练一练】
请尝试在 apply 传入的自定义函数中，根据组的某些特征返回相同长度但索引不同的 Series ，会报错吗？

In [503]:
gb = df.groupby(['Gender','Test_Number'])['Height']

In [504]:
gb.mean()

Gender  Test_Number
Female  1              159.865079
        2              158.187755
        3              159.565000
Male    1              172.314815
        2              175.705556
        3              173.283333
Name: Height, dtype: float64

In [505]:
def sy_test(x):
    res1=pd.Series([1,0],index=['a','b'])
    res2=pd.Series([0,0],index=['c','d'])
    if x.mean()>165:
        return res1
    else:
        return res2
    return

gb.apply(sy_test)
#没有报错

Gender  Test_Number   
Female  1            c    0
                     d    0
        2            c    0
                     d    0
        3            c    0
                     d    0
Male    1            a    1
                     b    0
        2            a    1
                     b    0
        3            a    1
                     b    0
Name: Height, dtype: int64

- DataFrame情况

行索引最内层在每个组原先 agg 的结果索引上，再加一层返回的 DataFrame 行索引，同时分组结果 DataFrame 的列索引和返回的 DataFrame 列索引一致

In [506]:
gb.apply(lambda x: pd.DataFrame(np.ones((2,2)),index = ['a','b'],columns=pd.Index([('w','x'),('y','z')])))

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0
Male   1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0

#### 【练一练】

请尝试在 apply 传入的自定义函数中，根据组的某些特征返回相同大小但列索引不同的 DataFrame ，会报错吗？如果只是行索引不同，会报错吗？

In [507]:
def sy_test(x):
    res1=pd.DataFrame(np.ones((2,2)),index=['a','b'],columns=pd.Index(['x','y']))
    res2=pd.DataFrame(np.ones((2,2)),index=['c','d'],columns=pd.Index(['u','v']))
    if x.mean()>165:
        return res1
    else:
        return res2
    return

gb.apply(sy_test)
#也没有报错

u    v    x    y
Gender Test_Number                      
Female 1           c  1.0  1.0  NaN  NaN
                   d  1.0  1.0  NaN  NaN
       2           c  1.0  1.0  NaN  NaN
                   d  1.0  1.0  NaN  NaN
       3           c  1.0  1.0  NaN  NaN
                   d  1.0  1.0  NaN  NaN
Male   1           a  NaN  NaN  1.0  1.0
                   b  NaN  NaN  1.0  1.0
       2           a  NaN  NaN  1.0  1.0
                   b  NaN  NaN  1.0  1.0
       3           a  NaN  NaN  1.0  1.0
                   b  NaN  NaN  1.0  1.0

#### 【练一练】
在 groupby 对象中还定义了 cov 和 corr 函数，从概念上说也属于跨列的分组处理。请利用之前定义的 gb 对象，使用apply函数实现与 gb.cov() 同样的功能并比较它们的性能。



In [508]:
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.cov()

Height     Weight
Gender                             
Female Height  25.542739  24.838146
       Weight  24.838146  29.224655
Male   Height  49.681137  47.803901
       Weight  47.803901  60.412648

In [509]:
gb.corr()

Height    Weight
Gender                           
Female Height  1.000000  0.889132
       Weight  0.889132  1.000000
Male   Height  1.000000  0.883847
       Weight  0.883847  1.000000

In [549]:
def my_cov(x):
    res=x[~np.isnan(x).any(axis=1)]#去除身高体重任意一个有nan值的行
    res_h=x[~np.isnan(x['Height'])]#去除身高有nan值的行
    res_w=x[~np.isnan(x['Weight'])]#去除体重有nan值的行
    res1=np.cov(res['Height'],res['Weight'])[1,0]
    res2=np.cov(res_h['Height'],res_h['Height'])[0,0]
    res3=np.cov(res_w['Weight'],res_w['Weight'])[0,0]
    temp=pd.DataFrame([[res2,res1],[res1,res3]],index=['Height','Weight'],columns=pd.Index(['Height','Weight']))
    return temp
gb.apply(my_cov)

Height     Weight
Gender                             
Female Height  25.542739  24.838146
       Weight  24.838146  29.224655
Male   Height  49.681137  47.803901
       Weight  47.803901  60.412648

# 五、练习
## EX1：汽车数据集
现有一份汽车数据集，其中 Brand, Disp., HP 分别代表汽车品牌、发动机蓄量、发动机输出。

In [635]:
df = pd.read_csv('D:\\datawhale\\joyful-pandas\\data\\car.csv')
df.head()

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63
3,Honda Civic 4,6635,Japan/USA,5.0,32,Small,2260,91,92
4,Mazda Protege 4,6599,Japan,5.0,32,Small,2440,113,103


1.先过滤出所属 Country 数超过2个的汽车，即若该汽车的 Country 在总体数据集中出现次数不超过2则剔除，再按 Country 分组计算价格均值、价格变异系数、该 Country 的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为 CoV 。

In [560]:
gp=df.groupby('Country')
gp.size()

Country
France        1
Germany       2
Japan        19
Japan/USA     7
Korea         3
Mexico        1
Sweden        1
USA          26
dtype: int64

In [573]:
p1=gp.filter(lambda x:x.shape[0]>2).groupby('Country')
gp1.agg({'Price':['mean',('CoV',lambda x:x.std()/x.mean()),('amount',lambda x:x.shape[0])]})

Price                 
                   mean       CoV amount
Country                                 
Japan      13938.052632  0.387429     19
Japan/USA  10067.571429  0.240040      7
Korea       7857.333333  0.243435      3
USA        12543.269231  0.203344     26

2.按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计 Price 的均值。

In [580]:
df.shape[0]
gplable=['First']*20+['Midle']*20+['Final']*20#用加号连接series
df.groupby(gplable)['Price'].mean()

Final    15420.65
First     9069.95
Midle    13356.40
Name: Price, dtype: float64

3.对类型 Type 分组，对 Price 和 HP 分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。

In [589]:
gp=df.groupby('Type')
res=gp.agg({'Price':['max','min'],'HP':['max','min']})
res.columns= res.columns.map(lambda x: (x[0]+'-'+x[1]))#res.columns.map(lambda x:'_'.join(x))
res

,Price-max,Price-min,HP-max,HP-min
Type,,,,
Compact,18900,9483,142,95
Large,17257,14525,170,150
Medium,24760,9999,190,110
Small,9995,5866,113,63
Sporty,13945,9410,225,92
Van,15395,12267,150,106


4.对类型 Type 分组，对 HP 进行组内的 min-max 归一化。

In [595]:
df.groupby('Type')['HP'].transform(lambda x:(x-x.min())/(x.max()-x.min())).head()

0    1.00
1    0.54
2    0.00
3    0.58
4    0.80
Name: HP, dtype: float64

5.对类型 Type 分组，计算 Disp. 与 HP 的相关系数。

In [606]:
df.groupby('Type').apply(lambda x:x['Disp.'].corr(x['HP']))

Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64

## EX2：实现transform函数
- groupby 对象的构造方法是 my_groupby(df, group_cols)
- 支持单列分组与多列分组
- 支持带有标量广播的 my_groupby(df)[col].transform(my_func) 功能
- pandas 的 transform 不能跨列计算，请支持此功能，即仍返回 Series 但 col 参数为多列
- 无需考虑性能与异常处理，只需实现上述功能，在给出测试样例的同时与 pandas 中的 transform 对比结果是否一致

In [613]:
df.head()

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63
3,Honda Civic 4,6635,Japan/USA,5.0,32,Small,2260,91,92
4,Mazda Protege 4,6599,Japan,5.0,32,Small,2440,113,103


In [646]:
class my_groupby:#用class详细规定一个函数
    def __init__(self, my_df, group_cols):#初始化定义self，传入df与列名,在这里是Type
        self.my_df = my_df.copy()
        self.groups = my_df[group_cols].drop_duplicates()#去除冗杂的,只保留第一次出现的行
        print(self.groups)
        if isinstance(self.groups, pd.Series):#如果取出的组为series，就转化成frame
            self.groups = self.groups.to_frame()
        self.group_cols = self.groups.columns.tolist()#取出的组名字为Type
        self.groups = {i: self.groups[i].values.tolist() for i in self.groups.columns}#建立字典，i为type
        print(self.groups)#将取出的组的每个元素与type做字典
        self.transform_col = None
    def __getitem__(self, col):
        self.pr_col = [col] if isinstance(col, str) else list(col)#列名放入self中
        return self
    def transform(self, my_func):#这种前后不含__的，可以通过class函数后面.来引用
        self.num = len(self.groups[self.group_cols[0]])#self.group_cols为输入的组名字type,num是分组的组数，在这里是六个
        L_order, L_value = np.array([]), np.array([])
        for i in range(self.num):#在每个组里面玩耍，group_df就是取出该组的数据
            group_df = self.my_df.reset_index().copy()
            for col in self.group_cols:#列表推导式
                group_df = group_df[group_df[col]==self.groups[col][i]]
            group_df = group_df[self.pr_col]
            if group_df.shape[1] == 1:
                group_df = group_df.iloc[:, 0]
            group_res = my_func(group_df)#对该组的运用自定义函数
            if not isinstance(group_res, pd.Series):
                group_res = pd.Series(group_res,index=group_df.index,name=group_df.name)
            L_order = np.r_[L_order, group_res.index]#索引
            L_value = np.r_[L_value, group_res.values]#值
        self.res = pd.Series(pd.Series(L_value, index=L_order).sort_index().values,index=self.my_df.reset_index().index, name=my_func.__name__)
        return self.res
                
def f(s):
    res = (s-s.min())/(s.max()-s.min())
    return res


my_groupby(df, 'Type')['Price'].transform(f).head()##单列分组

0       Small
13     Sporty
22    Compact
37     Medium
50      Large
53        Van
Name: Type, dtype: object
{'Type': ['Small', 'Sporty', 'Compact', 'Medium', 'Large', 'Van']}


0    0.733592
1    0.372003
2    0.109712
3    0.186244
4    0.177525
Name: f, dtype: float64

In [647]:
my_groupby(df, ['Type','Country'])['Price'].transform(f).head()##多列分组

       Type    Country
0     Small        USA
2     Small      Korea
3     Small  Japan/USA
4     Small      Japan
5     Small     Mexico
12    Small    Germany
13   Sporty        USA
17   Sporty      Japan
22  Compact    Germany
23  Compact        USA
27  Compact  Japan/USA
29  Compact      Japan
33  Compact     France
36  Compact     Sweden
37   Medium      Japan
38   Medium        USA
44   Medium      Korea
50    Large        USA
53      Van        USA
56      Van      Japan
{'Type': ['Small', 'Small', 'Small', 'Small', 'Small', 'Small', 'Sporty', 'Sporty', 'Compact', 'Compact', 'Compact', 'Compact', 'Compact', 'Compact', 'Medium', 'Medium', 'Medium', 'Large', 'Van', 'Van'], 'Country': ['USA', 'Korea', 'Japan/USA', 'Japan', 'Mexico', 'Germany', 'USA', 'Japan', 'Germany', 'USA', 'Japan/USA', 'Japan', 'France', 'Sweden', 'Japan', 'USA', 'Korea', 'USA', 'USA', 'Japan']}


0    1.000000
1    0.000000
2    0.000000
3    0.000000
4    0.196357
Name: f, dtype: float64

In [ ]:
my_groupby(df, 'Type')['Price'].transform(lambda x:x.mean()).head()